파이토치(PyTorch)의 nn.Embedding()은 단어 각각에 대해 정수가 맵핑된 입력에 대해서 임베딩 작업을 수행할 수 있게 해줍니다.

### 실제 RNN 은닉층을 추가하는 코드.
nn.RNN(input_size, hidden_size, batch_first=True)

텍스트 분류 관점에서 앞서 배운 RNN 코드의 timesteps와 input_dim, 그리고 hidden_size를 해석해보면 다음과 같습니다. (위의 코드에서는 바닐라 RNN을 사용했지만, RNN의 변형인 LSTM이나 GRU도 아래의 사항은 동일합니다.)

hidden_size = 출력의 크기(output_dim).  
timesteps = 시점의 수 = 각 문서에서의 단어 수.  
input_size = 입력의 크기 = 각 단어의 벡터 표현의 차원 수.  

---

# IMDB 리뷰 감성 분류하기(IMDB Movie Review Sentiment Analysis)

### 영화 사이트 imdb의 리뷰데이터
### 리뷰 긍정 = 1, 부정 = 0

In [31]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random
print(torch.__version__)

1.9.1+cpu


In [19]:
import torch
torch.cuda.is_available() 

False

In [3]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [105]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
# DEVICE = torch.device("cuda")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)
print("cpu와 cuda 중 다음 기기로 학습함:", USE_CUDA)

cpu와 cuda 중 다음 기기로 학습함: cpu
cpu와 cuda 중 다음 기기로 학습함: False


In [32]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
# 데이터셋이 순차적인 데이터셋임을 알 수 있도록 sequential 인자값으로 True를 명시
# batch_first는 신경망에 입력되는 텐서의 첫번째 차원값이 batch_size가 되도록 합니다
#  lower 변수를 통해 텍스트 데이터 속 모든 영문 알파벳이 소문자가 되도록 합니다
LABEL = data.Field(sequential=False, batch_first=True)
# 레이블은 단순한 클래스를 나타내는 숫자로 순차적인 데이터가 아니므로 False를 명시

In [35]:
# 전체 데이터를 훈련 데이터와 테스트 데이터를 8:2 비율로 나누기
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [36]:
print('trainset의 구성 요소 출력 : ', trainset.fields)

trainset의 구성 요소 출력 :  {'text': <torchtext.legacy.data.field.Field object at 0x0000025BB1846130>, 'label': <torchtext.legacy.data.field.Field object at 0x0000025BB18464C0>}


In [37]:
print(vars(trainset[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

## 3. 단어 집합 만들기 : 단어 집합이란 중복을 제거한 총 단어들의 집합을 의미

In [38]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
# min_freq는 학습 데이터에서 최소 5번 이상 등장한 단어만을 단어 집합에 추가하겠다는 의미
# 때 학습 데이터에서 5번 미만으로 등장한 단어는 Unknown이라는 의미에서 '<unk>'라는 토큰으로 대체됩니다.
LABEL.build_vocab(trainset)


In [39]:
vocab_size = len(TEXT.vocab)
n_classes = 2 # 긍정, 부정
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 46159
클래스의 개수 : 2


stoi로 단어와 각 단어의 정수 인덱스가 저장되어져 있는 딕셔너리 객체에 접근할 수 있습니다.

In [41]:
# print(TEXT.vocab.stoi) 
# defaultdict(<function _default_unk_index at 0x7fb279f3cc80>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5,
# 'zoe,': 46150, 'zombies"': 46151, 'zombies)': 46152, 'zombified': 46153, 'zone.<br': 46154, 'zoolander': 46155, 'zwick': 46156, '{the': 46157, 'émigré': 46158})

### 검증데이터 분리

In [42]:
trainset, valset = trainset.split(split_ratio=0.8)

## 토치텍스트는 모든 텍스트를 배치 처리하는 것을 지원
## 단어를 인덱스 번호로 대체하는 BucketIterator를 제공

In [43]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

In [50]:
print('훈련 데이터의 개수 : {}'.format(len(trainset)))
print('테스트 데이터의  개수 : {}'.format(len(testset)))
print('검증 데이터의 개수 : {}'.format(len(valset)))

훈련 데이터의 개수 : 20000
테스트 데이터의  개수 : 25000
검증 데이터의 개수 : 5000


In [44]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


In [98]:
batch = next(iter(train_iter)) # 첫번째 미니배치
print(iter(train_iter))
print(batch.text.shape)

<generator object Iterator.__iter__ at 0x0000025BBF380900>
torch.Size([64, 701])


In [99]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

## rnn 모델 구현

In [100]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [106]:

model = GRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 모델 훈련 함수

In [107]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

### 모델 평가 함수

In [108]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

### 모델 훈련시킨다

In [109]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.69 | val accuracy : 50.16
[Epoch: 2] val loss :  0.69 | val accuracy : 50.82
[Epoch: 3] val loss :  0.69 | val accuracy : 51.32
[Epoch: 4] val loss :  0.69 | val accuracy : 52.30
[Epoch: 5] val loss :  0.68 | val accuracy : 56.74
[Epoch: 6] val loss :  0.39 | val accuracy : 82.66
[Epoch: 7] val loss :  0.32 | val accuracy : 86.38
[Epoch: 8] val loss :  0.35 | val accuracy : 86.86
[Epoch: 9] val loss :  0.41 | val accuracy : 87.06
[Epoch: 10] val loss :  0.51 | val accuracy : 85.06


In [110]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.31 | 테스트 정확도: 86.86


# 4. 마지막 time step의 hidden state 가져오는 것 이해하기

In [111]:
# GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
# x, _ = self.gru(x, h_0)  
# (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
# h_t = x[:,-1,:] 

NameError: name 'self' is not defined

In [112]:
import torch
inputs = torch.rand(3, 4, 5) # 임의의 3차원 텐서 생성
print('텐서의 크기 :',inputs.shape)

텐서의 크기 : torch.Size([3, 4, 5])


In [113]:
print(inputs)

tensor([[[0.3214, 0.0583, 0.5687, 0.5495, 0.0235],
         [0.0072, 0.5005, 0.2986, 0.7326, 0.4478],
         [0.5289, 0.9558, 0.1968, 0.1341, 0.1031],
         [0.6537, 0.8530, 0.1503, 0.9026, 0.7126]],

        [[0.2545, 0.8816, 0.7804, 0.4833, 0.8863],
         [0.4772, 0.6559, 0.6713, 0.4474, 0.5454],
         [0.1302, 0.0150, 0.8998, 0.5673, 0.4571],
         [0.8940, 0.7837, 0.4960, 0.5229, 0.7837]],

        [[0.0728, 0.9748, 0.4680, 0.5153, 0.2849],
         [0.5245, 0.5134, 0.5415, 0.8048, 0.7301],
         [0.9094, 0.4021, 0.5698, 0.8958, 0.7573],
         [0.5089, 0.3722, 0.8606, 0.5076, 0.5181]]])


In [114]:
print(inputs[:, -1, :])

tensor([[0.6537, 0.8530, 0.1503, 0.9026, 0.7126],
        [0.8940, 0.7837, 0.4960, 0.5229, 0.7837],
        [0.5089, 0.3722, 0.8606, 0.5076, 0.5181]])


In [115]:
print('텐서의 크기 :',inputs[:, -1, :].shape)

텐서의 크기 : torch.Size([3, 5])
